In [5]:
#Importing the library
import tensorflow as tf
from d2l import tensorflow as d2l
import collections
import re

In [ ]:
#!pip install d2l==0.16.6 #==> if d2l is not working then setup the environment '''


In [6]:
#Reading the Dataset
#we load text from > The Time Machine <. This is a fairly small corpus of just over 30000 words
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

In [11]:
def read_time_machine():  #function made 
# Load the time machine dataset into a list of text lines 
    with open(d2l.download('time_machine'), 'r') as f: 
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]  #split the words into lower form

lines = read_time_machine()  


In [12]:
print(f'# text lines: {len(lines)}')  #length of lines
print(lines[0]) #first line
print(lines[10])  #tenth line

# text lines: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [13]:
#Tokenization
def tokenize(lines, token='word'):  
#Split text lines into word or character tokens
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unknown token type: ' + token)

In [14]:
tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [15]:
#Vocabulary
#build a dictionary, often called vocabulary 
class Vocab: 
#Vocabulary for text."""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
# Sort according to frequencies
        counter = count_corpus(tokens)  # count the unique tokens in all the documents from the training set named as corpus
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)
# The index for the unknown token is 0
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        uniq_tokens += [
            token for token, freq in self.token_freqs
            if freq >= min_freq and token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
        

In [16]:
def count_corpus(tokens):  #save
#Count token frequencies."""
    # Here `tokens` is a 1D list or 2D list
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # Flatten a list of token lists into a list of tokens
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [17]:
#We construct a vocabulary using the time machine dataset as the corpus. Then we print the first few frequent tokens with their indices
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [18]:
# convert each text line into a list of numerical indices
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [19]:
#Putting All Things Together
def load_corpus_time_machine(max_tokens=-1):  
#Return token indices and the vocabulary of the time machine dataset
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # Since each text line in the time machine dataset is not necessarily a
    # sentence or a paragraph, flatten all the text lines into a single list
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)

In [20]:
''' 
1) we tokenize text into characters, not words, to simplify the training in later sections 
2) corpus is a single list, not a list of token lists, since each text line in the time machine dataset is not necessarily a sentence or a paragraph '''

' \n1) we tokenize text into characters, not words, to simplify the training in later sections \n2) corpus is a single list, not a list of token lists, since each text line in the time machine dataset is not necessarily a sentence or a paragraph '